In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

import optuna
from sklearn.feature_selection import SelectKBest, chi2
import tensorflow as tf
from tensorflow import keras 
from scipy.sparse import csr_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgbm
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [2]:
%cd C:\Users\PAVAN\Downloads\mini project
train = pd.read_csv('train_data.csv', low_memory=False)

C:\Users\PAVAN\Downloads\mini project


In [3]:
test = pd.read_csv('test_data.csv', low_memory=False)
output = pd.DataFrame(test['MachineIdentifier'])

In [4]:
df = train.copy()

In [5]:
df.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,171,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2668.0,9124.0,4.0,5.0,2341.0,NaN,476940.0,HDD,299451.0,0,4096.0,Desktop,18.9,1440.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,88,20710.0,NaN,117.0,115,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,ExistsNotSet,1.0,1.0,Desktop,Windows.Desktop,1443.0,275890.0,4.0,5.0,2273.0,NaN,238475.0,UNKNOWN,227116.0,0,4096.0,MiniTower,18.5,1366.0,768.0,Desktop,NaN,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,17.0,64,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,355.0,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
2,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,18,37376.0,NaN,277.0,75,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,1443.0,331929.0,4.0,5.0,2500.0,NaN,476940.0,HDD,101900.0,0,6144.0,Portable,14.0,1366.0,768.0,Mobile,lion,0.000000e+00,10.0.17134.191,amd64,rs4_release,17134,191,Core,CORE,Update,8.0,31,FullAuto,0,IS_GENUINE,Retail,0.0,0.0,Retail,0.0,355.0,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1
3,000016191b897145d069102325cab760,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1094.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,97,13598.0,27.0,126.0,124,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,3799.0,340727.0,2.0,5.0,4324.0,NaN,114473.0,SSD,113671.0,0,8192.0,Desktop,21.5,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,16

In [6]:
df.shape

(7137943, 83)

## Pre Processing

In [7]:
df.dtypes

MachineIdentifier                                     object
ProductName                                           object
EngineVersion                                         object
AppVersion                                            object
AvSigVersion                                          object
IsBeta                                                 int64
RtpStateBitfield                                     float64
IsSxsPassiveMode                                       int64
DefaultBrowsersIdentifier                            float64
AVProductStatesIdentifier                            float64
AVProductsInstalled                                  float64
AVProductsEnabled                                    float64
HasTpm                                                 int64
CountryIdentifier                                      int64
CityIdentifier                                       float64
OrganizationIdentifier                               float64
GeoNameIdentifier       

In [9]:
df.drop(['MachineIdentifier','Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical','Census_IsWIMBootEnabled'],axis=1,inplace=True)
test.drop(['MachineIdentifier','Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical','Census_IsWIMBootEnabled'],axis=1,inplace=True)

In [10]:
# For float value columns

float_columns = ['Census_PrimaryDiskTotalCapacity','Census_SystemVolumeTotalCapacity','Census_TotalPhysicalRAM','Census_InternalPrimaryDiagonalDisplaySizeInInches','Census_InternalBatteryNumberOfCharges']

df['Census_PrimaryDiskTotalCapacity'] //= 10000
test['Census_PrimaryDiskTotalCapacity'] //= 10000
df[['Census_SystemVolumeTotalCapacity','Census_TotalPhysicalRAM']] //= 1000
test[['Census_PrimaryDiskTotalCapacity','Census_SystemVolumeTotalCapacity','Census_TotalPhysicalRAM']] //= 1000
df['Census_InternalPrimaryDiagonalDisplaySizeInInches'] //= 1
test['Census_InternalPrimaryDiagonalDisplaySizeInInches'] //= 1
df['Census_InternalBatteryNumberOfCharges'] //= 100000000
test['Census_InternalBatteryNumberOfCharges'] //= 100000000

# Replace null values
for column in float_columns:
    df[column].fillna(df[column].median(),inplace=True)
    test[column].fillna(df[column].median(),inplace=True)
    
# Outliers
std = df[float_columns].std()
mean = df[float_columns].mean()

z_scores = (df[float_columns] - mean)/std
filtered_entries = (np.abs(z_scores) <= 3).all(axis=1)
df = df[filtered_entries]

z_scores = (test[float_columns] - mean)/std
for column in float_columns:
    test.loc[z_scores[column] > 3, column] = mean[column] + 3*std[column]
    test.loc[z_scores[column] < -3, column] = mean[column] - 3*std[column]
    df[column] = df[column].astype('float32')
    test[column] = test[column].astype('float32')

In [11]:
y = df['HasDetections']
df.drop('HasDetections',axis=1,inplace=True)

## One-Hot Encoding

In [7]:
# For int value columns

for column in df.select_dtypes('int64').columns:
    df[column] = df[column].astype('int16')
    
int_columns = list(set(df.select_dtypes('number')) - set(float_columns))

for column in int_columns:
    if df[column].dtype == 'int16' and df[column].min() == 0 and df[column].max() == 1:
        continue
    for item in df[column].value_counts().items():
        if item[1] > df.shape[0]*(0.05):
            col_name = str(column)+'_'+str(item[0])
            df[col_name] = np.where(df[column]==item[0],1,0)
            df[col_name] = df[col_name].astype('int16')
            
            test[col_name] = np.where(test[column]==item[0],1,0)
            test[col_name] = test[col_name].astype('int16')
            
        else:
            break
    df.drop(column,axis=1,inplace=True)
    test.drop(column,axis=1,inplace=True)

In [8]:
# For object value columns

for column in df.select_dtypes('object').columns:
    for item in df[column].value_counts().items():
        if item[1] > df.shape[0]*(0.05):
            col_name = str(column)+'_'+str(item[0])
            df[col_name] = np.where(df[column]==item[0],1,0)
            df[col_name] = df[col_name].astype('int16')
            
            test[col_name] = np.where(test[column]==item[0],1,0)
            test[col_name] = test[col_name].astype('int16')
            
        else:
            break
    df.drop(column,axis=1,inplace=True)
    test.drop(column,axis=1,inplace=True)

In [9]:
df.shape

(6934954, 153)

In [10]:
# No more null values
for item in df.isna().sum().items():
    if item[1] > 0:
        print(item)

## Label Encoder

In [51]:
# For int value columns (Categorical)

for column in df.select_dtypes('int64').columns:
    df[column] = df[column].astype('int32')
    
int_columns = list(set(df.select_dtypes('number')) - set(float_columns))

for column in int_columns:
    if df[column].dtype == 'int32' and df[column].min() == 0 and df[column].max() == 1:
        continue
    else:
        label_encoder = LabelEncoder()
        label_encoder.fit(pd.concat([df[column], test[column]]))
        df[column] = label_encoder.transform(df[column])
        test[column] = label_encoder.transform(test[column])
        df[column] = df[column].astype('int32')
        test[column] = test[column].astype('int32')

In [52]:
# For object value columns

for column in df.select_dtypes('object').columns:
    label_encoder = LabelEncoder()
    label_encoder.fit(pd.concat([df[column], test[column]]))
    df[column] = label_encoder.transform(df[column])
    test[column] = label_encoder.transform(test[column])
    df[column] = df[column].astype('int32')
    test[column] = test[column].astype('int32')

In [53]:
df.shape

(6934954, 79)

In [54]:
for item in df.isna().sum().items():
    if item[1] > 0:
        print(item)

In [55]:
df.dtypes

ProductName                                          int32
EngineVersion                                        int32
AppVersion                                           int32
AvSigVersion                                         int32
IsBeta                                               int32
RtpStateBitfield                                     int32
IsSxsPassiveMode                                     int32
DefaultBrowsersIdentifier                            int32
AVProductStatesIdentifier                            int32
AVProductsInstalled                                  int32
AVProductsEnabled                                    int32
HasTpm                                               int32
CountryIdentifier                                    int32
CityIdentifier                                       int32
OrganizationIdentifier                               int32
GeoNameIdentifier                                    int32
LocaleEnglishNameIdentifier                          int

## Model

In [57]:
df.head()

,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,4,65,59,7789,0,4,0,2001,21753,1,1,1,28,81046,15,33,168,0,1,0,9,4,4,269,6,1,0,2,0,98,21,1,1,2,1,1660,4423,3,3,1779,3,47,0,299,0,4,23,18,1,77,42,357,0,10,98,38,16,15,6,25,104,5,0,1,2,2,0,6,2,405,24431,0,0,0,0,0,0,9
1,4,65,59,7711,0,4,0,2001,21753,1,1,1,87,13282,49,113,112,0,1,0,9,4,4,269,6,1,0,2,0,98,7,1,1,2,1,869,132037,3,3,1721,3,23,2,227,0,4,35,18,1,77,42,361,0,10,98,51,16,15,6,16,58,2,0,1,1,2,0,6,2,219,14032,0,0,0,0,0,0,2
2,4,65,59,7670,0,4,0,2001,21753,1,1,1,17,24149,49,272,72,0,1,0,9,11,4,269,4,1,0,2,0,98,13,1,1,8,1,869,164355,3,3,1895,3,47,0,101,0,6,40,14,3,52,0,359,0,10,98,42,2,2,7,7,28,2,0,1,2,0,0,6,0,219,13905,0,0,0,0,0,0,0
3,4,65,59,7590,0,4,0,2001,21753,1,1,1,96,8901,23,122,121,0,1,0,9,4,4,269,6,1,0,2,0,98,13,1,1,2,1,2354,169304,1,3,3266,3,11,1,113,0,8,23,21,1,77,42,357,0,10,98,38,16,15,6,17,66,2,0,1,2,0,0,6,0,54,35216,0,0,0,0,0,0,14
4,4,65,59,7972,0,4,0,2001,17589,2,1,1,77,51401,49,85,85,0,1,0,9,11,4,269,4,1,0,2,0,98,21,1,1,8,1,2354,99690,1,0,557,3,47,0,458,0,4,37,17,3,77,0,357,0,10,98,38,2,2,2,13,44,2,0,1,2,2,0,6,2,346,43437,1,0,0,0,0,0,9


In [30]:
feautures = SelectKBest(chi2, k='all')
df = feautures.fit_transform(df, y)
test = feautures.transform(test)

In [32]:
# df_sparse = csr_matrix(df, dtype='float32')
# test_sparse = csr_matrix(test, dtype='float32')

In [69]:
def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['XGB'])    
    
    limit = round(0.98*df.shape[0])
    val_size = round(0.99*df.shape[0])
    
#     if classifier_name == 'LGBM':
#         max_depth = trial.suggest_int('LGBM max_depth', 2, 14)
#         learning_rate = trial.suggest_float('LGBM learning rate', 1e-4, 100, log=True)
#         n_estimators = trial.suggest_int('LGBM n_estimators', 50, 300)
#         num_leaves = trial.suggest_int('LGBM num_leaves', 3, 2**12-1)
        
#         classifier_obj = lgbm.LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, 
#                                         max_depth=max_depth, num_leaves=num_leaves, random_state=0, device='gpu')
#         eval_set=[(df[limit:val_size], y[limit:val_size])]
#         classifier_obj.fit(df[:limit], y[:limit], eval_set=eval_set, eval_metric='auc',
#                            callbacks = [lgbm.callback.early_stopping(stopping_rounds=100)])
#         score = roc_auc_score(y[val_size:], classifier_obj.predict(df[val_size:]))
#         return score
        
#     if classifier_name == 'Logistic':
#         C = trial.suggest_float('C', 1e-6, 1e2, log=True)
#         classifier_obj = LogisticRegression(C=C, max_iter=7000000, random_state=0)
#         classifier_obj.fit(df_sparse[:val_size], y[:val_size])       
#         score = roc_auc_score(y[val_size:], classifier_obj.predict(df_sparse[val_size:]))
#         return score
        
#     elif classifier_name == 'RandomForest':
#         rf_n_estimators = trial.suggest_int('rf_n_estimators', 50, 300)
#         rf_max_depth = trial.suggest_int('rf_max_depth', 20, 100)
#         classifier_obj = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=rf_n_estimators)
#         score = model_selection.cross_val_score(classifier_obj, df_sparse, y, n_jobs=3, cv=3, scoring='roc_auc')
    
    if classifier_name == 'XGB':
        xgb_n_estimators = trial.suggest_int('xgb_n_estimators', 50, 200)
        xgb_max_depth = trial.suggest_int('xgb_max_depth', 2, 14)
        classifier_obj = XGBClassifier(max_depth=xgb_max_depth, n_estimators=xgb_n_estimators, use_label_encoder=False, 
                                       tree_method='gpu_hist')
        eval_set=[(df[limit:val_size], y[limit:val_size])]
        classifier_obj.fit(df[:limit], y[:limit], eval_set=eval_set, eval_metric='auc', early_stopping_rounds = 5)       
        score = roc_auc_score(y[val_size:], classifier_obj.predict(df[val_size:]))
        return score
    
#     elif classifier_name == 'GNB':
#         gnb_var_smoothing = trial.suggest_float('gnb_var_smoothing', 1e-9, 1, log=True)
#         classifier_obj = GaussianNB(var_smoothing=gnb_var_smoothing)
#         score = model_selection.cross_val_score(classifier_obj, df_sparse.toarray(), y, n_jobs=4, cv=2, scoring='roc_auc')

    accuracy = score.mean()
    return accuracy

In [70]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2021-12-22 02:30:08,076] A new study created in memory with name: no-name-03c54ead-a861-41d9-8b65-79a8b68edbb8
C:\Users\PAVAN\anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

[0]	validation_0-auc:0.67175
[1]	validation_0-auc:0.67698
[2]	validation_0-auc:0.68089
[3]	validation_0-auc:0.68461
[4]	validation_0-auc:0.68754
[5]	validation_0-auc:0.68946
[6]	validation_0-auc:0.69045
[7]	validation_0-auc:0.69237
[8]	validation_0-auc:0.69353
[9]	validation_0-auc:0.69505
[10]	validation_0-auc:0.69612
[11]	validation_0-auc:0.69715
[12]	validation_0-auc:0.69796
[13]	validation_0-auc:0.70044
[14]	validation_0-auc:0.70152
[15]	validation_0-auc:0.70217
[16]	validation_0-auc:0.70333
[17]	validation_0-auc:0.70365
[18]	validation_0-auc:0.70426
[19]	validation_0-auc:0.70468
[20]	validation_0-auc:0.70560
[21]	validation_0-auc:0.70615
[22]	validation_0-auc:0.70673
[23]	validation_0-auc:0.70789
[24]	validation_0-auc:0.70818
[25]	validation_0-auc:0.70908
[26]	validation_0-auc:0.70938
[27]	validation_0-auc:0.71006
[28]	validation_0-auc:0.71042
[29]	validation_0-auc:0.71069
[30]	validation_0-auc:0.71104
[31]	validation_0-auc:0.71161
[32]	validation_0-auc:0.71182
[33]	validation_0-au

[37]	validation_0-auc:0.73059
[38]	validation_0-auc:0.73063
[39]	validation_0-auc:0.73071
[40]	validation_0-auc:0.73070
[41]	validation_0-auc:0.73072
[42]	validation_0-auc:0.73073
[43]	validation_0-auc:0.73119
[44]	validation_0-auc:0.73122
[45]	validation_0-auc:0.73123
[46]	validation_0-auc:0.73131
[47]	validation_0-auc:0.73137
[48]	validation_0-auc:0.73149
[49]	validation_0-auc:0.73148
[50]	validation_0-auc:0.73163
[51]	validation_0-auc:0.73178
[52]	validation_0-auc:0.73178
[53]	validation_0-auc:0.73198
[54]	validation_0-auc:0.73198
[55]	validation_0-auc:0.73197
[56]	validation_0-auc:0.73203
[57]	validation_0-auc:0.73213
[58]	validation_0-auc:0.73215
[59]	validation_0-auc:0.73216
[60]	validation_0-auc:0.73219
[61]	validation_0-auc:0.73220
[62]	validation_0-auc:0.73218
[63]	validation_0-auc:0.73213
[64]	validation_0-auc:0.73212
[65]	validation_0-auc:0.73205
[I 2021-12-22 02:37:32,573] Trial 2 finished with value: 0.6689074187583997 and parameters: {'classifier': 'XGB', 'xgb_n_estimator

[9]	validation_0-auc:0.67105
[10]	validation_0-auc:0.67167
[11]	validation_0-auc:0.67460
[12]	validation_0-auc:0.67486
[13]	validation_0-auc:0.67495
[14]	validation_0-auc:0.67560
[15]	validation_0-auc:0.67606
[16]	validation_0-auc:0.67685
[17]	validation_0-auc:0.67768
[18]	validation_0-auc:0.67834
[19]	validation_0-auc:0.67949
[20]	validation_0-auc:0.67952
[21]	validation_0-auc:0.67990
[22]	validation_0-auc:0.68041
[23]	validation_0-auc:0.68107
[24]	validation_0-auc:0.68157
[25]	validation_0-auc:0.68329
[26]	validation_0-auc:0.68380
[27]	validation_0-auc:0.68434
[28]	validation_0-auc:0.68447
[29]	validation_0-auc:0.68478
[30]	validation_0-auc:0.68498
[31]	validation_0-auc:0.68534
[32]	validation_0-auc:0.68642
[33]	validation_0-auc:0.68720
[34]	validation_0-auc:0.68737
[35]	validation_0-auc:0.68751
[36]	validation_0-auc:0.68760
[37]	validation_0-auc:0.68800
[38]	validation_0-auc:0.68830
[39]	validation_0-auc:0.68864
[40]	validation_0-auc:0.68891
[41]	validation_0-auc:0.68911
[42]	valida

[23]	validation_0-auc:0.70283
[24]	validation_0-auc:0.70307
[25]	validation_0-auc:0.70382
[26]	validation_0-auc:0.70390
[27]	validation_0-auc:0.70440
[28]	validation_0-auc:0.70453
[29]	validation_0-auc:0.70519
[30]	validation_0-auc:0.70573
[31]	validation_0-auc:0.70624
[32]	validation_0-auc:0.70653
[33]	validation_0-auc:0.70675
[34]	validation_0-auc:0.70707
[35]	validation_0-auc:0.70773
[36]	validation_0-auc:0.70806
[37]	validation_0-auc:0.70820
[38]	validation_0-auc:0.70845
[39]	validation_0-auc:0.70876
[40]	validation_0-auc:0.70911
[41]	validation_0-auc:0.70920
[42]	validation_0-auc:0.70982
[43]	validation_0-auc:0.71009
[44]	validation_0-auc:0.71029
[45]	validation_0-auc:0.71077
[46]	validation_0-auc:0.71103
[47]	validation_0-auc:0.71119
[48]	validation_0-auc:0.71122
[49]	validation_0-auc:0.71136
[50]	validation_0-auc:0.71149
[51]	validation_0-auc:0.71164
[52]	validation_0-auc:0.71184
[53]	validation_0-auc:0.71196
[54]	validation_0-auc:0.71231
[55]	validation_0-auc:0.71244
[56]	valid

[15]	validation_0-auc:0.72226
[16]	validation_0-auc:0.72294
[17]	validation_0-auc:0.72368
[18]	validation_0-auc:0.72397
[19]	validation_0-auc:0.72458
[20]	validation_0-auc:0.72512
[21]	validation_0-auc:0.72550
[22]	validation_0-auc:0.72602
[23]	validation_0-auc:0.72663
[24]	validation_0-auc:0.72676
[25]	validation_0-auc:0.72737
[26]	validation_0-auc:0.72752
[27]	validation_0-auc:0.72783
[28]	validation_0-auc:0.72779
[29]	validation_0-auc:0.72819
[30]	validation_0-auc:0.72819
[31]	validation_0-auc:0.72840
[32]	validation_0-auc:0.72868
[33]	validation_0-auc:0.72884
[34]	validation_0-auc:0.72892
[35]	validation_0-auc:0.72913
[36]	validation_0-auc:0.72913
[37]	validation_0-auc:0.72917
[38]	validation_0-auc:0.72930
[39]	validation_0-auc:0.72954
[40]	validation_0-auc:0.72954
[41]	validation_0-auc:0.72955
[42]	validation_0-auc:0.72976
[43]	validation_0-auc:0.72993
[44]	validation_0-auc:0.73006
[45]	validation_0-auc:0.73017
[46]	validation_0-auc:0.73019
[47]	validation_0-auc:0.73028
[48]	valid

In [71]:
print(study.best_value)
print(study.best_params)

0.6698451194011912
{'classifier': 'XGB', 'xgb_n_estimators': 86, 'xgb_max_depth': 13}


In [ ]:
model = LogisticRegression(C=57.55, max_iter=7000000)

In [ ]:
model.fit(df_sparse,y)

In [118]:
model = XGBClassifier(max_depth=11, n_estimators=150, use_label_encoder=False, tree_method='gpu_hist')

In [119]:
limit = round(0.8*df.shape[0])
val_size = round(0.99*df.shape[0])
eval_set=[(df[limit:val_size], y[limit:val_size])]
model.fit(df[:limit], y[:limit], eval_set=eval_set, eval_metric='auc', early_stopping_rounds=10)       
score = roc_auc_score(y[val_size:], model.predict(df[val_size:]))
score

[0]	validation_0-auc:0.69492
[1]	validation_0-auc:0.69904
[2]	validation_0-auc:0.70234
[3]	validation_0-auc:0.70504
[4]	validation_0-auc:0.70723
[5]	validation_0-auc:0.70899
[6]	validation_0-auc:0.71047
[7]	validation_0-auc:0.71187
[8]	validation_0-auc:0.71438
[9]	validation_0-auc:0.71608
[10]	validation_0-auc:0.71664
[11]	validation_0-auc:0.71766
[12]	validation_0-auc:0.71860
[13]	validation_0-auc:0.71964
[14]	validation_0-auc:0.72046
[15]	validation_0-auc:0.72092
[16]	validation_0-auc:0.72197
[17]	validation_0-auc:0.72230
[18]	validation_0-auc:0.72318
[19]	validation_0-auc:0.72395
[20]	validation_0-auc:0.72449
[21]	validation_0-auc:0.72480
[22]	validation_0-auc:0.72552
[23]	validation_0-auc:0.72613
[24]	validation_0-auc:0.72648
[25]	validation_0-auc:0.72686
[26]	validation_0-auc:0.72713
[27]	validation_0-auc:0.72743
[28]	validation_0-auc:0.72770
[29]	validation_0-auc:0.72788
[30]	validation_0-auc:0.72846
[31]	validation_0-auc:0.72861
[32]	validation_0-auc:0.72871
[33]	validation_0-au

0.6716487778308952

In [120]:
output['HasDetections'] = model.predict(test)

In [121]:
output.to_csv('submission.csv',index=False)